In [6]:
import matplotlib.pyplot as plt
import xlsxwriter
import numpy as np
import pandas as pd
import pandas.io.sql
import os
import pdfkit
import pytz 
from datetime import datetime 
import base64
import pymssql
import psycopg2 as pg
from jinja2 import Environment, FileSystemLoader

host        = '192.168.1.13'
database    = 'igu_live'
user        = 'sa'
password    = 'B1admin'


host2        = "192.168.9.235"
database2    = "igportal"
user2        = "invoice"
password2    = "invoice" 

conn2 = pg.connect("host="+ host2 + " dbname="+  database2 + " user="+ user2 + " password="+ password2 + "")
 
conn = pymssql.connect(host=host, user=user, password=password, database=database)

msg_sql= """
            SELECT DISTINCT 
                        '20200707' 'Date To',
                        T0.DOCENTRY ,
                        T3.DOCSTATUS, 
                        CONVERT(vARCHAR,T1.docduedate,112) DOCDATE,
                        t3.docnum,
                        T2.BEGINSTR+ CONVERT(VARCHAR,T1.DOCNUM) ord_no ,
                        T1.CARDCODE, 
                        T1.SHIPTOCODE , 
                        t6.groupname memo,
                        T1.CARDNAME ,
                        T3.NUMATCARD,
                        T4.U_SlsEmpName, 
                        T3.DocTotal,
                        T1.COMMENTS 
                FROM DLN1 T0 
                    INNER JOIN ORDR T1 ON T0.BASEENTRY = T1.DOCENTRY AND T0.BASETYPE=17 
                    INNER JOIN OCRD T5 ON T1.cardcode = t5.cardcode 
                    INNER JOIN OCRG T6 ON T5.groupcode = t6.groupcode 
                    INNER JOIN OSLP  T4 ON T5.SlpCode=T4.SlpCode
                    INNER JOIN NNM1 T2 ON T1.[Series] = T2.[Series] AND T0.[TargetType] not in (13,15)
                    INNER JOIN ODLN T3 ON T0.DOCENTRY = T3.DOCENTRY 
                WHERE   
                         T3.DOCSTATUS ='O'
                    AND CONVERT(VARCHAR,T1.docduedate,112)>='20161231'
                    and CONVERT(VARCHAR,T1.docduedate,112)<='20200807'
        """

pandas.options.display.float_format = '{:,.2f}'.format

#msg_sql = """ exec [dbo].[IGU_AGING_AR] '20200531','IGU' """
data = pandas.io.sql.read_sql(msg_sql,conn)

msg_sql2= """
            select  a.s_canceled , 
                    a.numatcard , 
                    to_char(a.docdate,'yyyyMMdd') ,
                    a.ord_no ,
                    a.cardcode ,
                    a.cardname ,
                    a.shiptocode ,
                    to_char(a.sap_createdate,'yyyyMMdd') createdate , 
                    a.doctime ,
                    case a.posisi 
                    when 'who' then 'Process in Wh Admin'
                    when 'whi' then 'deliver To Accounting'
                    when 'acin' then 'Process in Accounting'
                    end as status,
                    a.shiptocode,
                    a.remarks,
                    a.status,
                    coalesce(a.notes,'') notes,
                    a.state done_status,
                    a.notes_log ,
                    a.date_release tgl_naik,
                    a.customer_group ,
                    a.salesperson ,
                    a.salesgroup ,
                    a.cancel_reason

            from sp_do_list a  where a.ord_no in ('""" + "','".join(data["ord_no"].tolist()) + """')
        """


data2 = pandas.io.sql.read_sql(msg_sql2,conn2)

data2.to_excel("doBelumInvoice.xlsx")

print(msg_sql2)

#data3 =pandas.merge()





            select  a.s_canceled , 
                    a.numatcard , 
                    to_char(a.docdate,'yyyyMMdd') ,
                    a.ord_no ,
                    a.cardcode ,
                    a.cardname ,
                    a.shiptocode ,
                    to_char(a.sap_createdate,'yyyyMMdd') createdate , 
                    a.doctime ,
                    case a.posisi 
                    when 'who' then 'Process in Wh Admin'
                    when 'whi' then 'deliver To Accounting'
                    when 'acin' then 'Process in Accounting'
                    end as status,
                    a.shiptocode,
                    a.remarks,
                    a.status,
                    coalesce(a.notes,'') notes,
                    a.state done_status,
                    a.notes_log ,
                    a.date_release tgl_naik,
                    a.customer_group ,
                    a.salesperson ,
                    a.salesgroup ,
                   